In [ ]:
%load_ext autoreload
%autoreload 2

# basic imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats


def get_data(dataset_name, arm):
    dataset = pd.read_csv("../../Datasets/" + dataset_name + ".csv")
    instances = sorted(dataset["instance"].unique())
    all_arm_index_list = dataset["arm_index"].unique()
    valid_arm_index_list = [item for item in all_arm_index_list if item >= 0]
    number_of_arms = len(valid_arm_index_list)
    number_of_trails = len(dataset["repetition"].unique())
    horizon_time = len(dataset["iteration"].unique())

    d_stats = (
        dataset[dataset["iteration"] == 1]
        .groupby(["instance", "repetition"])["loss"]
        .agg(["mean", "std"])
    )
    dataset = dataset.merge(
        d_stats, on=["instance", "repetition"], suffixes=("", "_stats")
    )
    dataset["norm_loss"] = 1 / (
        1 + np.exp(-(dataset["loss"] - dataset["mean"]) / (dataset["std"] + 0.01))
    )
    dataset = dataset.drop(columns=["mean", "std"])
    # def min_max_norm(x):
    #     print(x)
    #     return 1 / (1 + np.exp(-(x - np.mean(d)) / (np.std(d) + 0.01)))

    # dataset["norm_loss"] = dataset.groupby("instance")["loss"].transform(min_max_norm)

    df = dataset[(dataset["arm_index"] >= 0) & (dataset["arm_index"] == arm)]
    df = df[["instance", "arm_index", "repetition", "iteration", "norm_loss"]]
    real_data = df.sort_values(by=["instance", "arm_index", "repetition", "iteration"])[
        "norm_loss"
    ].values.reshape(len(instances), 1, number_of_trails, horizon_time)
    return real_data

In [ ]:
import numpy as np
import torch
import pandas as pd
from scipy.stats import skewnorm


def non_stationary_skew_normal_dist(sigma1, sigma2, seq_len):
    skewness = np.random.uniform(-100.0, -20.0)
    loc1 = np.random.uniform(0.0, 1.0)
    scale1 = np.random.uniform(0.0, sigma1)

    quantile1 = skewnorm.cdf(0.0, a=skewness, loc=loc1, scale=scale1)
    quantile2 = skewnorm.cdf(1.0, a=skewness, loc=loc1, scale=scale1)
    samples_1 = skewnorm.ppf(
        np.random.uniform(quantile1, quantile2, size=seq_len),
        a=skewness,
        loc=loc1,
        scale=scale1,
    )

    skewness = np.random.uniform(-100.0, -20.0)
    scale2 = np.random.uniform(sigma2 * (1 - loc1), sigma2)
    quantile1 = skewnorm.cdf(0.0, a=skewness, loc=1.0, scale=scale2)
    quantile2 = skewnorm.cdf(1.0, a=skewness, loc=1.0, scale=scale2)
    samples_3 = skewnorm.ppf(
        np.random.uniform(quantile1, quantile2, size=seq_len),
        a=skewness,
        loc=1.0,
        scale=scale2,
    )
    samples_3 = np.sort(samples_3)
    mixed_samples = samples_1 * samples_3
    mixed_samples[mixed_samples > 1.0] = 1.0
    mixed_samples[mixed_samples < 0.0] = 0.0
    return mixed_samples


def get_batch_func(
    batch_size,
    seq_len,
    num_features=1,
    device="cpu",
    hyperparameters=None,
    noisy_target=True,
    num_outputs=1,
    sigma1=0.1,
    sigma2=0.1,
    to_torch=False,
    **_,
):
    xs = np.zeros((seq_len, batch_size, num_features))
    ys = np.zeros((seq_len, batch_size, num_outputs))
    ys_noisy = np.zeros((seq_len, batch_size, num_outputs))

    for i in range(batch_size):
        xs[:, i, 0] = np.arange(1, seq_len + 1)
        data = non_stationary_skew_normal_dist(sigma1, sigma2, seq_len)
        ys[:, i, 0] = np.maximum.accumulate(data, axis=0)
        ys_noisy[:, i, 0] = np.maximum.accumulate(
            data + np.random.normal(0.0, 0.001, seq_len), axis=0
        )
        if num_outputs > 1:
            ys[:, i, 1] = data
    if to_torch:
        xs = torch.from_numpy(xs.astype(np.float32))
        ys = torch.from_numpy(ys.astype(np.float32))
        ys_noisy = torch.from_numpy(ys_noisy.astype(np.float32))
        return xs.to(device), ys.to(device), ys.to(device)
    else:
        return xs, ys, ys

In [ ]:
import numpy as np

def error_from_trends(A_data, B_data, ci=0.95):
    print("Calculating error from trends...", A_data.shape , B_data.shape)
    mean_A = A_data.mean(axis=0)
    mean_B = B_data.mean(axis=0)
    return np.sqrt(np.mean((mean_A - mean_B)**2))


In [ ]:
import os


def plot_real_vs_synthetic(
    dataset_name, T, arm_names, arm, sigma1, sigma2, samples=1000
):
    real_datas = []
    real_datas.append(get_data(dataset_name, arm))

    real_data = []
    for d in real_datas:
        arr = d.copy().reshape(
            d.shape[0] * d.shape[1] * d.shape[2],
            d.shape[3],
        )
        real_data.append(arr)
    real_data = np.concatenate(real_data)

    # Assuming `data` is your (1200, 250) array
    first_observations = 200
    column_0 = np.max(real_data[:, 0:first_observations], axis=1)  # Mean across the first dimension
    # Create 20 bins
    num_bins = 10
    bins = np.linspace(0, 1, num_bins + 1)

    grouped_data = {i: [] for i in range(num_bins)}
    bin_indices = np.digitize(column_0, bins) - 1
    for i, bin_idx in enumerate(bin_indices):
        if 0 <= bin_idx < num_bins:
            grouped_data[bin_idx].append(real_data[i])

    # Optional: Convert lists to arrays
    grouped_data_real_data = {
        k: np.array(v) for k, v in grouped_data.items() if len(v) > 0
    }
    grouped_data_synthetic_data = {}
    num_sample = samples
    _, synthetic_data, _ = get_batch_func(
        num_sample, T, num_outputs=2, sigma1=sigma1, sigma2=sigma2
    )
    synthetic_series = synthetic_data[:, :, 1].T

    column_0 = np.max(synthetic_series[:, 0:first_observations], axis=1)
    bin_indices = np.digitize(column_0, bins) - 1
    grouped_data = {i: [] for i in range(num_bins)}
    for i, bin_idx in enumerate(bin_indices):
        if 0 <= bin_idx < num_bins:
            grouped_data[bin_idx].append(synthetic_series[i])

    # Optional: Convert lists to arrays
    grouped_data_synthetic_data = {
        k: np.array(v) for k, v in grouped_data.items() if len(v) > 0
    }
    all_a = []
    all_b = []
    for i in grouped_data_real_data.keys():
        if i not in grouped_data_synthetic_data.keys():
            continue
        start_bin = bins[i] 
        b_min = 0
        b_max =  bins[i+1]
        #max(grouped_data_real_data[i].max(axis=0).mean(), grouped_data_synthetic_data[i].max(axis=0).mean())
        a = ( np.maximum.accumulate(grouped_data_synthetic_data[i], axis=1) - b_min)/ ( b_max)
        b = (np.maximum.accumulate(grouped_data_real_data[i], axis=1) - b_min ) / ( b_max)
        all_a.append(a)
        all_b.append(b)

    all_a = np.concatenate(all_a, axis=0)
    all_b = np.concatenate(all_b, axis=0)

    def get_median_and_ci(matrix, lower=25, upper=75):
        median = np.median(matrix, axis=0)
        lower_ci = np.percentile(matrix, lower, axis=0)
        upper_ci = np.percentile(matrix, upper, axis=0)
        return median, lower_ci, upper_ci

    # Group A
    median_a, low_a, high_a = get_median_and_ci(all_a)

    # Group B
    median_b, low_b, high_b = get_median_and_ci(all_b)

    error = error_from_trends(all_a, all_b)

    time = np.arange(T)

    plt.figure(figsize=(5, 4))
    plt.rcParams.update({"font.size": 12})

    # Plot group A with shaded std
    plt.plot(
        time,
        median_a,
        color="blue",
        label="Synth($\\sigma_1=" + str(sigma1) + ", \\sigma_2=" + str(sigma2) + "$)",
        linewidth=3,
    )
    plt.fill_between(time, low_a, high_a, color="blue", alpha=0.2)

    # Plot group B with shaded std
    plt.plot(
        time,
        median_b,
        color="orange",
        label= dataset_name.split("_")[0] + "-" + arm_names[arm],  # Real(" + dataset_name.split("_")[0] + "-" + arm_names[arm] + ")",
        linewidth=3,
    )
    plt.fill_between(time, low_b, high_b, color="orange", alpha=0.2)

    plt.xlabel("Time Steps")
    plt.ylabel("Scaled performance")
    plt.legend(title=f"RMSE = {error:.3f}", loc="lower right")
    plt.tight_layout()
    path = "./figures/paper_prior_selection_" + dataset_name + "/"
    if not os.path.exists(path):
        os.makedirs(path)
    plt.savefig(
        path + arm_names[arm] + "_" + str(sigma1) + "_" + str(sigma2) + ".pdf",
        bbox_inches="tight",
    )
    plt.close()

In [ ]:
dataset_name = "Reshuffling"
dataset_name = "YaHPOGym_3"
#dataset_name = "TabRepoRaw_8"

max_retries = 5
T = 200
if dataset_name == "Reshuffling":
    arm_names = ["MLP", "Logreg", "CatBoost", "XGBoost"]
    T = 250
if dataset_name == "YaHPOGym_3":
    arm_names = ["AKNN", "GLMNet", "Ranger", "RPart", "SVM", "XGBoost"]
if dataset_name == "TabRepoRaw_8":
    arm_names = [
        "CatBoost",
        "ExtraTrees",
        "LightGBM",
        "NeuralNet(FastAI)",
        "NeuralNet(Torch)",
        "RandomForest",
        "XGBoost",
    ]

sigma1_sigma2 = [(0.1, 0.001), (0.2, 0.01), (0.2, 0.1)]

for sigma1, sigma2 in sigma1_sigma2:
    for arm in range(len(arm_names)):
        print("arm_name", arm, arm_names[arm], "sigma", sigma1, sigma2)
        for attempt in range(max_retries):
            try:
                # Your code here (e.g., network request, file read, etc.)
                result = plot_real_vs_synthetic(
                    dataset_name, T, arm_names, arm, sigma1, sigma2
                )
                break  # Success, exit the loop
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt == max_retries - 1:
                    print("Max retries reached. Exiting.")
                    break